In [ ]:
from transformers import DataCollatorWithPadding
from datasets import load_dataset

#載入資料
dataset = load_dataset("csv",data_files="./ChnSentiCorp_htl_all.csv",split="train")
dataset


Dataset({
    features: ['label', 'review'],
    num_rows: 7766
})

In [4]:
#清理資料
cleared_dataset = dataset.filter(lambda item: item['review'] is not None)
cleared_dataset

Filter:   0%|          | 0/7766 [00:00<?, ? examples/s]

Dataset({
    features: ['label', 'review'],
    num_rows: 7765
})

In [33]:
from transformers import AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained('google-bert/bert-base-chinese')
def process_tokenizer(example:dict)->dict:
    '''
    只要先分詞,不要現在轉成tensor,轉成tensor,由DataCollator來作
    '''
    tokenized = tokenizer(example['review'],max_length=128,truncation=True)
    tokenized['label'] = example['label']
    return tokenized


tokenizer_dataset = cleared_dataset.map(function=process_tokenizer,remove_columns=cleared_dataset.column_names)
tokenizer_dataset[:2]
    

Map:   0%|          | 0/7765 [00:00<?, ? examples/s]

{'label': [1, 1],
 'input_ids': [[101,
   6655,
   7431,
   2335,
   3763,
   1062,
   6662,
   6733,
   6818,
   117,
   852,
   3221,
   1062,
   769,
   2900,
   4850,
   679,
   2205,
   117,
   1963,
   3362,
   3221,
   107,
   5918,
   7380,
   5221,
   107,
   4638,
   6282,
   117,
   3298,
   7478,
   2382,
   7937,
   4214,
   119,
   2456,
   6359,
   4500,
   1162,
   4638,
   6662,
   5221,
   119,
   2791,
   7279,
   6733,
   4158,
   5080,
   1606,
   119,
   102],
  [101,
   1555,
   1243,
   1920,
   2414,
   2791,
   8024,
   2791,
   7279,
   2523,
   1920,
   8024,
   2414,
   3300,
   100,
   2184,
   8024,
   3146,
   7768,
   2697,
   6221,
   5195,
   4089,
   2179,
   2669,
   679,
   7097,
   106,
   102]],
 'token_type_ids': [[0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   

In [ ]:
collator = DataCollatorWithPadding(tokenizer=tokenizer,return_tensors='pt')
collator

DataCollatorWithPadding(tokenizer=BertTokenizerFast(name_or_path='google-bert/bert-base-chinese', vocab_size=21128, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
), padding=True, max_length=None, pad_to_multiple_of=None, return_tensors='pt')

In [ ]:
from torch.utils.data import DataLoader
d1 = DataLoader(tokenizer_dataset, batch_size=4, collate_fn=collator, shuffle=True)

next(enumerate(d1))[1] #可以發現已經轉成tensor

{'input_ids': tensor([[  101,  2769,   947,  3176,  8126,  3189,   678,  1286,  2537,  6832,
          3172,  2225,  1962,  5844,  1854,  6983,  2421,   889,  1168,  2356,
          1281,  1057,   857,  6857,  2157,  7613,  2421,  8024,  7613,  2421,
          1762,   671,   943,   100,   100,  6174,  7481,  8024,  3760,   889,
          6882,  4638,  6917,  4696,   679,  6210,  2533,  5543,  2523,  3175,
           912,  4638,  2823,  1168,   511,  6794,  4415,  4638,  6862,  2428,
          2523,  2571,  8024,  2706,  2428,   738,  2523,  1962,  8024,  1920,
          5147,   126,  1146,  7132,  2218,  3018,  2137,   511,   889,  1168,
          2791,  7279,  2527,  6366,  2769,   947,  3300,  7953,   679,  6900,
          2746,  8024,  1728,  4158,  4696,  4638,  2523,  2207,  8024,  2215,
          1071,  3221,  1190,   857,  6882,  6832,  3172,  2225,  1962,  5844,
          1854,  6983,  2421,  8024,  2791,  7279,  7481,  4948,  1920,  5147,
          1372,  3300,  8108,  2398,  

In [ ]:
num = 0
for batch in d1:
    print(batch['input_ids'].size())
    num += 1
    if num > 10:
        break
#每個bach有4個,長度超過128只有128,小於128會依據實際的長度

torch.Size([4, 128])
torch.Size([4, 128])
torch.Size([4, 128])
torch.Size([4, 128])
torch.Size([4, 128])
torch.Size([4, 124])
torch.Size([4, 79])
torch.Size([4, 128])
torch.Size([4, 128])
torch.Size([4, 128])
torch.Size([4, 128])
